In [2]:
import muon as mu
import numpy as np
import pandas as pd
import torch
import matplotlib.pyplot as plt

/home/wuxinchao/software/miniconda3/envs/mofa2/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
sp_data_path = "/home/wuxinchao/data/st_cite_data/"
sp_mudata = mu.read_h5mu(sp_data_path + "sp_mudata.h5mu")

In [5]:
sp_mudata

MuData object with n_obs × n_vars = 8628 × 25828
  uns:	'mofa'
  obsm:	'spatial', 'X_mofa'
  varm:	'LFs'
  2 modalities
    rna:	8628 x 25818
      var:	'mt', 'highly_variable', 'highly_variable_rank', 'means', 'variances', 'variances_norm', 'mean', 'std'
      uns:	'hvg', 'log1p', 'neighbors', 'pca', 'umap'
      obsm:	'X_pca', 'X_umap', 'spatial'
      varm:	'PCs'
      layers:	'counts'
      obsp:	'connectivities', 'distances'
    prot:	8628 x 10
      obsm:	'spatial'

In [4]:
mu.tl.mofa(sp_mudata, use_obs='union',
           n_factors=15, convergence_mode='medium',
           outfile="models/CLL_union.hdf5")


        #########################################################
        ###           __  __  ____  ______                    ### 
        ###          |  \/  |/ __ \|  ____/\    _             ### 
        ###          | \  / | |  | | |__ /  \ _| |_           ### 
        ###          | |\/| | |  | |  __/ /\ \_   _|          ###
        ###          | |  | | |__| | | / ____ \|_|            ###
        ###          |_|  |_|\____/|_|/_/    \_\              ###
        ###                                                   ### 
        ######################################################### 
       
 
        
Loaded view='rna' group='group1' with N=8628 samples and D=25818 features...
Loaded view='prot' group='group1' with N=8628 samples and D=10 features...



Model options:
- Automatic Relevance Determination prior on the factors: True
- Automatic Relevance Determination prior on the weights: True
- Spike-and-slab prior on the factors: False
- Spike-and-slab prior on the weights: T

MuData object with n_obs × n_vars = 8628 × 25828
  uns:	'mofa'
  obsm:	'spatial', 'X_mofa'
  varm:	'LFs'
  2 modalities
    rna:	8628 x 25818
      var:	'mt', 'highly_variable', 'highly_variable_rank', 'means', 'variances', 'variances_norm', 'mean', 'std'
      uns:	'hvg', 'log1p', 'neighbors', 'pca', 'umap'
      obsm:	'X_pca', 'X_umap', 'spatial'
      varm:	'PCs'
      layers:	'counts'
      obsp:	'connectivities', 'distances'
    prot:	8628 x 10
      obsm:	'spatial'